In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime

from itertools import combinations
import plotly.express as px

<built-in method __dir__ of sqlite3.Connection object at 0x7fbfd4dfda40>


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'AVAXUSDT'
interval = '1m'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >= ;2020-10-01;2023-07-31 12:00:00;
start_test_date = '2023-07-31'  # train < and test >=

stop_loss = 1.0
# regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 120
normalize = True
use_gpu = False
tune_model = False
apply_combination_features = False

In [3]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=True, start_date=start_train_date)
data = data[data['open_time'] >= start_train_date]
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data = utils.truncate_data_to_train(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145606 entries, 2023-08-27 00:00:00 to 2023-12-06 02:45:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     145606 non-null  datetime64[ns]
 1   open                          145606 non-null  float32       
 2   high                          145606 non-null  float32       
 3   low                           145606 non-null  float32       
 4   close                         145606 non-null  float32       
 5   volume                        145606 non-null  float32       
 6   close_time                    145606 non-null  int64         
 7   quote_asset_volume            145606 non-null  float32       
 8   number_of_trades              145606 non-null  int16         
 9   taker_buy_base_asset_volume   145606 non-null  float32       
 10  taker_buy_quote_asset_volume  145606 non-null 

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol
ix_open_time,,,,,,,,,,,,
2023-08-27 00:00:00,2023-08-27 00:00:00,10.100000,10.100000,10.090000,10.090000,1782.640015,1693094459999,17991.302734,51,1385.130005,13980.426758,AVAXUSDT
2023-08-27 00:01:00,2023-08-27 00:01:00,10.100000,10.100000,10.090000,10.090000,25.280001,1693094519999,255.099899,4,2.470000,24.947001,AVAXUSDT
2023-08-27 00:02:00,2023-08-27 00:02:00,10.100000,10.100000,10.090000,10.090000,53.470001,1693094579999,539.536987,2,2.470000,24.947001,AVAXUSDT
2023-08-27 00:03:00,2023-08-27 00:03:00,10.100000,10.100000,10.100000,10.100000,350.170013,1693094639999,3536.717041,21,350.170013,3536.717041,AVAXUSDT
2023-08-27 00:04:00,2023-08-27 00:04:00,10.100000,10.100000,10.090000,10.090000,608.609985,1693094699999,6146.931641,24,605.659973,6117.166016,AVAXUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-06 02:41:00,2023-12-06 02:41:00,26.110001,26.170000,26.100000,26.139999,8781.330078,1701830519999,229475.968750,556,5492.720215,143543.062500,AVAXUSDT
2023-12-06 02:42:00,2023-12-06 02:42:00,26.139999,26.250000,26.120001,26.219999,6414.189941,1701830579999,167975.390625,630,4014.419922,105124.210938,AVAXUSDT
2023-12-06 02:43:00,2023-12-06 02:43:00,26.230000,26.290001,26.200001,26.230000,8131.509766,1701830639999,213497.343750,512,4049.219971,106328.218750,AVAXUSDT


In [4]:
data = calc_utils.calc_RSI(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145606 entries, 2023-08-27 00:00:00 to 2023-12-06 02:45:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     145606 non-null  datetime64[ns]
 1   open                          145606 non-null  float32       
 2   high                          145606 non-null  float32       
 3   low                           145606 non-null  float32       
 4   close                         145606 non-null  float32       
 5   volume                        145606 non-null  float32       
 6   close_time                    145606 non-null  int64         
 7   quote_asset_volume            145606 non-null  float32       
 8   number_of_trades              145606 non-null  int16         
 9   taker_buy_base_asset_volume   145606 non-null  float32       
 10  taker_buy_quote_asset_volume  145606 non-null 

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi
ix_open_time,,,,,,,,,,,,,
2023-08-27 00:00:00,2023-08-27 00:00:00,10.100000,10.100000,10.090000,10.090000,1782.640015,1693094459999,17991.302734,51,1385.130005,13980.426758,AVAXUSDT,NaN
2023-08-27 00:01:00,2023-08-27 00:01:00,10.100000,10.100000,10.090000,10.090000,25.280001,1693094519999,255.099899,4,2.470000,24.947001,AVAXUSDT,NaN
2023-08-27 00:02:00,2023-08-27 00:02:00,10.100000,10.100000,10.090000,10.090000,53.470001,1693094579999,539.536987,2,2.470000,24.947001,AVAXUSDT,NaN
2023-08-27 00:03:00,2023-08-27 00:03:00,10.100000,10.100000,10.100000,10.100000,350.170013,1693094639999,3536.717041,21,350.170013,3536.717041,AVAXUSDT,NaN
2023-08-27 00:04:00,2023-08-27 00:04:00,10.100000,10.100000,10.090000,10.090000,608.609985,1693094699999,6146.931641,24,605.659973,6117.166016,AVAXUSDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-06 02:41:00,2023-12-06 02:41:00,26.110001,26.170000,26.100000,26.139999,8781.330078,1701830519999,229475.968750,556,5492.720215,143543.062500,AVAXUSDT,52.852436
2023-12-06 02:42:00,2023-12-06 02:42:00,26.139999,26.250000,26.120001,26.219999,6414.189941,1701830579999,167975.390625,630,4014.419922,105124.210938,AVAXUSDT,57.593605
2023-12-06 02:43:00,2023-12-06 02:43:00,26.230000,26.290001,26.200001,26.230000,8131.509766,1701830639999,213497.343750,512,4049.219971,106328.218750,AVAXUSDT,58.160007


In [5]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145606 entries, 2023-08-27 00:00:00 to 2023-12-06 02:45:00
Data columns (total 17 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     145606 non-null  datetime64[ns]
 1   open                          145606 non-null  float32       
 2   high                          145606 non-null  float32       
 3   low                           145606 non-null  float32       
 4   close                         145606 non-null  float32       
 5   volume                        145606 non-null  float32       
 6   close_time                    145606 non-null  int64         
 7   quote_asset_volume            145606 non-null  float32       
 8   number_of_trades              145606 non-null  int16         
 9   taker_buy_base_asset_volume   145606 non-null  float32       
 10  taker_buy_quote_asset_volume  145606 non-null 

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_120p,ema_120p_diff,ema_200p,ema_200p_diff
ix_open_time,,,,,,,,,,,,,,,,,
2023-08-27 00:00:00,2023-08-27 00:00:00,10.100000,10.100000,10.090000,10.090000,1782.640015,1693094459999,17991.302734,51,1385.130005,13980.426758,AVAXUSDT,NaN,NaN,NaN,NaN,NaN
2023-08-27 00:01:00,2023-08-27 00:01:00,10.100000,10.100000,10.090000,10.090000,25.280001,1693094519999,255.099899,4,2.470000,24.947001,AVAXUSDT,NaN,NaN,NaN,NaN,NaN
2023-08-27 00:02:00,2023-08-27 00:02:00,10.100000,10.100000,10.090000,10.090000,53.470001,1693094579999,539.536987,2,2.470000,24.947001,AVAXUSDT,NaN,NaN,NaN,NaN,NaN
2023-08-27 00:03:00,2023-08-27 00:03:00,10.100000,10.100000,10.100000,10.100000,350.170013,1693094639999,3536.717041,21,350.170013,3536.717041,AVAXUSDT,NaN,NaN,NaN,NaN,NaN
2023-08-27 00:04:00,2023-08-27 00:04:00,10.100000,10.100000,10.090000,10.090000,608.609985,1693094699999,6146.931641,24,605.659973,6117.166016,AVAXUSDT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-06 02:41:00,2023-12-06 02:41:00,26.110001,26.170000,26.100000,26.139999,8781.330078,1701830519999,229475.968750,556,5492.720215,143543.062500,AVAXUSDT,52.852436,26.046328,0.359635,25.739290,1.556800
2023-12-06 02:42:00,2023-12-06 02:42:00,26.139999,26.250000,26.120001,26.219999,6414.189941,1701830579999,167975.390625,630,4014.419922,105124.210938,AVAXUSDT,57.593605,26.049198,0.655687,25.744074,1.848680
2023-12-06 02:43:00,2023-12-06 02:43:00,26.230000,26.290001,26.200001,26.230000,8131.509766,1701830639999,213497.343750,512,4049.219971,106328.218750,AVAXUSDT,58.160007,26.052187,0.682525,25.748909,1.868392


In [6]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145407 entries, 2023-08-27 03:19:00 to 2023-12-06 02:45:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     145407 non-null  datetime64[ns]
 1   open                          145407 non-null  float32       
 2   high                          145407 non-null  float32       
 3   low                           145407 non-null  float32       
 4   close                         145407 non-null  float32       
 5   volume                        145407 non-null  float32       
 6   close_time                    145407 non-null  int64         
 7   quote_asset_volume            145407 non-null  float32       
 8   number_of_trades              145407 non-null  int16         
 9   taker_buy_base_asset_volume   145407 non-null  float32       
 10  taker_buy_quote_asset_volume  145407 non-null 

In [7]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145407 entries, 2023-08-27 03:19:00 to 2023-12-06 02:45:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   open_time                     145407 non-null  datetime64[ns]
 1   open                          145407 non-null  float32       
 2   high                          145407 non-null  float32       
 3   low                           145407 non-null  float32       
 4   close                         145407 non-null  float32       
 5   volume                        145407 non-null  float32       
 6   close_time                    145407 non-null  int64         
 7   quote_asset_volume            145407 non-null  float32       
 8   number_of_trades              145407 non-null  int16         
 9   taker_buy_base_asset_volume   145407 non-null  float32       
 10  taker_buy_quote_asset_volume  145407 non-null 

In [8]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

,open_time,perc
status,,
ESTAVEL,77420,0.532437
LONG_1.0,36183,0.248839
SHORT_1.0,31804,0.218724


In [9]:
train_data, test_data = utils.split_train_test_data(data)
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15806 entries, 2023-08-27 03:19:00 to 2023-09-07 02:44:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     15806 non-null  datetime64[ns]
 1   open                          15806 non-null  float32       
 2   high                          15806 non-null  float32       
 3   low                           15806 non-null  float32       
 4   close                         15806 non-null  float32       
 5   volume                        15806 non-null  float32       
 6   close_time                    15806 non-null  int64         
 7   quote_asset_volume            15806 non-null  float32       
 8   number_of_trades              15806 non-null  int16         
 9   taker_buy_base_asset_volume   15806 non-null  float32       
 10  taker_buy_quote_asset_volume  15806 non-null  float32      

In [10]:
perc_data_label = train_data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(train_data)
perc_data_label

,open_time,perc
status,,
ESTAVEL,12745,0.806339
LONG_1.0,1443,0.091294
SHORT_1.0,1618,0.102366


In [11]:
perc_data_label = test_data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(test_data)
perc_data_label

,open_time,perc
status,,
ESTAVEL,64675,0.499032
LONG_1.0,34740,0.268053
SHORT_1.0,30186,0.232915


In [12]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# text_numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
#text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
#text_numeric_features = f'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ema_{int(times_regression_PnL)}p'
#text_numeric_features = f'quote_asset_volume,number_of_trades,ema_{int(times_regression_PnL)}p,ema_200p'
text_numeric_features = f'close,taker_buy_quote_asset_volume,ema_{times_regression_PnL}p'
#text_numeric_features2 = f'volume,taker_buy_base_asset_volume,ema_{int(times_regression_PnL)}p'

numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
if apply_combination_features:
	for size in range(1, len(numeric_features) + 1): 
		comb = map(list, combinations(numeric_features, size))	
		for c in comb:
			res = ''
			for j in c:
				res += f'{j},'
			combination_numeric_features.append(res[0:len(res) - 1])
else:
	combination_numeric_features = [text_numeric_features]

print(f'Combination Numeric Features: \n{combination_numeric_features}')

Numeric Features: ['close', 'taker_buy_quote_asset_volume', 'ema_120p'] - size: 3

Combination Numeric Features: 
['close,taker_buy_quote_asset_volume,ema_120p']


In [13]:
from datetime import datetime
import os
from imblearn.under_sampling import InstanceHardnessThreshold, RepeatedEditedNearestNeighbours, AllKNN


#iht = InstanceHardnessThreshold(estimator=KNeighborsClassifier())

# fix_imbalance_method: condensednearestneighbour, editednearestneighborus, repeatededitednearestneighbours, allknn, instancehardnessthreshold, nearmiss, neighbourhoodcleaningrule, onesidedselection, randomundersampler, tomeklinks, randomoversampler, smote, smotenc, smoten, adasyn, borderlinesmote, kmeanssmote, svmsmote, smoteenn, smotetomek.
# 'condensednearestneighbour,editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# 'smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
#imbalance_list = 'editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
imbalance_list = ['instancehardnessthreshold']
simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'
print(f'simulation_results_filename: {simulation_results_filename}')

df_resultado_simulacao = pd.DataFrame()
has_simulation_file = os.path.exists(simulation_results_filename)
if has_simulation_file:
  df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')

setup = None

for aux_numeric_features in combination_numeric_features:
  experiement = ClassificationExperiment()
  features = []
  features += ['open_time', myenv.label]
  features += aux_numeric_features.split(',')
  print(f'features: {features}')
  for fix_imbalance_method in imbalance_list:
    print(f'fix_imbalance_method: {fix_imbalance_method}')
    fim = fix_imbalance_method
    match(fix_imbalance_method):
      case 'repeatededitednearestneighbours':
        fim = RepeatedEditedNearestNeighbours(kind_sel='all', max_iter=100, n_jobs=20, n_neighbors=3, sampling_strategy='auto')
      case 'instancehardnessthreshold':
        fim = InstanceHardnessThreshold(cv=5, estimator=None, n_jobs=20, random_state=123, sampling_strategy='auto')
      case 'allknn':
        fim = AllKNN(allow_minority=False, kind_sel='all', n_jobs=20, n_neighbors=3, sampling_strategy='auto')

    setup = experiement.setup(
        data=train_data[features].copy(),
        train_size=myenv.train_size,
        target=myenv.label,
        numeric_features=aux_numeric_features.split(','),
        date_features=['open_time'],
        create_date_columns=["hour", "day", "month"],
        data_split_shuffle=False,
        data_split_stratify=False,
        fix_imbalance=True,
        fix_imbalance_method=fim,
        remove_outliers=True,
        fold_strategy='timeseries', 
        fold=3,
        session_id=123,
        normalize=normalize,
        use_gpu=True,
        verbose=False,
        n_jobs=20,
        log_experiment=False,
    )

    best = setup.create_model('lightgbm', n_jobs=20, return_train_score=False, verbose=False)
    if tune_model:
      best = setup.tune_model(best)  
    final_model = setup.finalize_model(best, model_only=True) 
    
    ajusted_test_data = test_data[features].drop(columns=[myenv.label])
    ajusted_test_data.info()

    df_final_predict = setup.predict_model(final_model, data=ajusted_test_data, raw_score=False, verbose=False)

    res_score = None
    if test_data is not None:
      df_final_predict[myenv.label] = test_data[myenv.label]
      df_final_predict['_score'] = df_final_predict['prediction_label'] == df_final_predict[myenv.label]

      print(f'Score Mean: {df_final_predict["_score"].mean()}')
      print(f'Score Group: \n{df_final_predict[[myenv.label, "_score"]].groupby(myenv.label).mean()}')
      res_score = df_final_predict[[myenv.label, '_score']].groupby(myenv.label).mean().copy()    

    data_simul = utils.simule_trading_crypto2(df_final_predict, df_final_predict.index.min(), df_final_predict.index.max(), 100.0)
    data_simul.to_csv(simulation_results_filename, sep=';', index=False)
    print(data_simul)

simulation_results_filename: ./resultado_simulacao_AVAXUSDT_1m.csv
features: ['open_time', 'status', 'close', 'taker_buy_quote_asset_volume', 'ema_120p']
fix_imbalance_method: instancehardnessthreshold
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> 

AttributeError: 'numpy.float64' object has no attribute 'to_csv'